### Creation of model based on previously created training and testing data
* Currently supports only linear models for continuous data. 
* Exports data into "models" folder

In [187]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [188]:

import numpy as np
from sklearn.model_selection import train_test_split
from joblib import dump
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error


# from sklearn import preprocessing
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier



In [189]:
mod_val = 1112

In [190]:
data = np.load(f"data/training{mod_val}.npz", allow_pickle=True)
X = data['X']
y = data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [191]:
# import matplotlib.pyplot as plt
# plt.plot(y_train)
# plt.show()
# y_train.shape

In [192]:
# plt.plot(X_train)

In [193]:
y_train[y_train==0] = 0.1
y_test[y_test==0] = 0.1

In [194]:
z_train = np.log(y_train)
z_test = np.log(y_test)

In [195]:
linreg = LinearRegression()
ridge = Ridge()
lasso = Lasso()
models = [linreg, ridge, lasso]
titles = ["linear regression", "ridge", "lasso"]


for i in range(len(models)):

    models[i].fit(X_train, z_train)
    print(f"{titles[i]} accuracy: {models[i].score(X_test, z_test)}")



linear regression accuracy: 0.9121901480304273
ridge accuracy: 0.910553289158941
lasso accuracy: 0.5819312815238171


In [196]:
y_pred = np.exp(linreg.predict(X_test))

for i in range(len(y_test)):
    print(f"predicted: {y_pred[i]}")
    print(f"actual: {y_test[i]}")
    print("~~")

predicted: 88.4633509998541
actual: 94.4639
~~
predicted: 53.184935590092486
actual: 62.1767
~~
predicted: 159.10516704010385
actual: 149.514
~~
predicted: 128.61327905607166
actual: 143.369
~~
predicted: 96.31803716584491
actual: 95.137
~~
predicted: 91.56783848399915
actual: 108.231
~~
predicted: 60.997620634546216
actual: 65.0563
~~
predicted: 72.50016268203737
actual: 80.8165
~~
predicted: 98.35019309024239
actual: 115.066
~~
predicted: 80.45353642128777
actual: 60.8389
~~
predicted: 27.88072509556489
actual: 30.4963
~~
predicted: 126.75060667706087
actual: 139.081
~~
predicted: 28.83620399289603
actual: 22.944
~~
predicted: 64.72788953501615
actual: 57.4623
~~
predicted: 33.63135597664401
actual: 26.3479
~~
predicted: 35.954464079739815
actual: 38.0043
~~
predicted: 146.9494741664061
actual: 156.452
~~
predicted: 32.32751465553967
actual: 34.859
~~
predicted: 69.10608241840991
actual: 41.2329
~~
predicted: 10.13775070396905
actual: 13.1039
~~
predicted: 7.387915350299588
actual: 2

In [197]:
dump(linreg, f'models/linreg{mod_val}.joblib')

['models/linreg1112.joblib']

In [198]:
# lab = preprocessing.LabelEncoder()
# y_train_enc = lab.fit_transform(y_train)
# y_test_enc = lab.fit_transform(y_test)

In [199]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train_enc)

# y_pred_rf = rf.predict(X_test)
# r2 = r2_score(y_test, y_pred_rf)
# mse = mean_squared_error(y_test, y_pred_rf)
# print(f"r2 score: {r2}")
# print(f"mean squared error: {mse}")

In [200]:
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_train, y_train_enc)

# y_pred_knn = knn.predict(X_test)
# r2 = r2_score(y_test, y_pred_knn)
# mse = mean_squared_error(y_test, y_pred_knn)
# print(f"r2 score: {r2}")
# print(f"mean squared error: {mse}")